# Lab | Random Forests

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_csv('files_for_lab/categorical.csv')
df2 = pd.read_csv('files_for_lab/numerical.csv')
df3 = pd.read_csv('files_for_lab/target.csv')
data = pd.concat([df1, df2, df3], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [3]:
data.shape

(95412, 339)

In [4]:
data.isna().sum()

STATE       0
CLUSTER     0
HOMEOWNR    0
GENDER      0
DATASRCE    0
           ..
HPHONE_D    0
RFA_2F      0
CLUSTER2    0
TARGET_B    0
TARGET_D    0
Length: 339, dtype: int64

In [5]:
data.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0,0.0
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,0,0.0
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10

In [6]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

### Apply the Random Forest algorithm to predict the TARGET_B. Please note that this column suffers from class imbalance. Fix the class imbalance using upsampling.

In [7]:
# upsampling the minority 
from sklearn.utils import resample
df_majority = data[data['TARGET_B'] == 0]
df_minority = data[data['TARGET_B'] == 1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # reproducible results
data1 = pd.concat([df_majority, df_minority_upsampled], axis=0)
data1.shape

(181138, 339)

In [8]:
y = data1[['TARGET_B','TARGET_D']]
X = data1.drop(['TARGET_B', 'TARGET_D'], axis = 1)
X.shape

(181138, 337)

In [9]:
high_cardinality_columns = []
for column in X.columns:
    unique_values = X[column].nunique()
    if unique_values > 10:  # You can adjust the threshold (e.g., 10) based on your preference
        high_cardinality_columns.append((column, unique_values))
if high_cardinality_columns:
    print("Columns with high cardinality:")
    for column, cardinality in high_cardinality_columns:
        print(f"{column}: {cardinality} unique values")
else:
    print("No columns with high cardinality found.")
    
X1 = X.loc[:, ~X.columns.isin(high_cardinality_columns)]

Columns with high cardinality:
STATE: 12 unique values
CLUSTER: 53 unique values
ODATEW_YR: 15 unique values
ODATEW_MM: 12 unique values
DOB_YR: 96 unique values
DOB_MM: 12 unique values
MINRDATE_YR: 20 unique values
MINRDATE_MM: 12 unique values
MAXRDATE_YR: 18 unique values
MAXRDATE_MM: 12 unique values
LASTDATE_MM: 12 unique values
FIRSTDATE_YR: 26 unique values
FIRSTDATE_MM: 12 unique values
TCODE: 55 unique values
AGE: 97 unique values
HIT: 75 unique values
MALEMILI: 95 unique values
MALEVET: 89 unique values
VIETVETS: 95 unique values
WWIIVETS: 100 unique values
LOCALGOV: 55 unique values
STATEGOV: 65 unique values
FEDGOV: 55 unique values
POP901: 9906 unique values
POP902: 4786 unique values
POP903: 5698 unique values
POP90C1: 100 unique values
POP90C2: 100 unique values
POP90C3: 100 unique values
POP90C4: 81 unique values
POP90C5: 81 unique values
ETH1: 100 unique values
ETH2: 100 unique values
ETH3: 85 unique values
ETH4: 96 unique values
ETH5: 100 unique values
ETH6: 21 uniqu

In [10]:
X1

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

numericalX1 = X1.select_dtypes(np.number)
categoricalX1 = X1.select_dtypes(include=['object'])
display(categoricalX1.head())

for col in categoricalX1.columns:
    print(col, categoricalX1[col].nunique())
print()

encoder = OneHotEncoder(drop='first').fit(categoricalX1)
encoded_categorical = encoder.transform(categoricalX1).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(), index=categoricalX1.index)



# Concatenate numericalX and encoded_categorical along axis=1
X1 = pd.concat([numericalX1, encoded_categorical], axis=1)

# Note: We need to do train/test split before downsampling, and then only downsample the training set - Why?
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=0)

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,IL,H,F,L,E,C,T
1,CA,H,M,L,G,A,S
2,NC,U,M,L,E,C,R
3,CA,U,F,L,E,C,R
4,FL,H,F,L,F,A,S


STATE 12
HOMEOWNR 2
GENDER 3
RFA_2R 1
RFA_2A 4
GEOCODE2 4
DOMAIN_A 5



In [12]:
X1_train.head()

,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
42043,45,2,2,92,1,17,6,95,2,95,1,96,3,92,10,1,81.000000,3,9,0,0,38,24,46,14,4,4,1,9656,2704,3747,0,0,99,50,50,83,0,17,0,1,0,0,0,0,0,0,0,1,0,0,0,36,47,50,38,49,52,27,39,39,21,9,19,19,13,15,13,12,16,17,33,20,14,32,13,30,24,38,38,23,10,4,60,9,10,21,168,252,82,81,2,5,3,2,2,2,0,524,598,2,2,72,28,46,54,92,34,72,58,24,83,17,11,2,8,15,17,4,16,44,23,0,0,0,1,3,9,21,54,0,6,34,49,49,13,2,4,1,11,17,5,3,2,0,2,14,43,0.0,381.0,676.0,194,226,249,286,9927,39,23,16,13,7,1,1,1,0,30,24,18,15,9,2,1,1,0,36,13,33,20,44,56,12,73,11,0,0,0,0,0,10,5,3,12,12,18,3,41,58,64,51,56,47,68,51,70,92,10,12,12,2,12,13,0,1,18,5,11,6,4,4,4,0,8,12,3,2,2,23,4,3,7,2,8,8,5,8,14,4,4,13,61,5,0,120,9,13,39,17,6,10,4,2,23,2,19,3,0,0,0,19,38,2,25,17,45,8,1,4,1,18,0,0,3,1,5,3,0,0,0,3,0,1,59,96,0,0,4,91,54,14,6,19,2,12,23,52,66,34,0,0,0,44,10,17,0,28,16,81,15,96,92,9,2,7,8,18,47,6,13,78.0,20,8,3.0,5.0,5.0,7,3.900000,81538,0,4,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
68510,5,2,2,90,1,24,5,90,4,93,10,95,12,90,4,28,74.000000,4,9,0,0,29,46,29,10,2,3,6,1276,332,490,99,0,0,49,51,89,0,1,5,22,0,1,1,1,1,0,0,10,1,4,8,37,43,45,38,46,49,20,37,37,26,12,20,21,16,10,12,9,19,12,30,21,18,32,13,27,24,33,42,24,10,4,52,11,10,27,177,259,89,87,2,10,8,6,0,0,0,2247,2272,7,7,78,22,98,2,8,31,68,54,24,82,18,6,1,5,17,23,8,14,56,19,0,0,0,72,93,98,99,99,4,7,45,52,52,14,5,8,2,0,15,4,3,0,73,82,96,98,4480.0,13.0,803.0,428,446,469,518,20390,10,16,17,18,24,6,7,0,1,2,13,22,17,34,5,6,0,2,34,1,48,2,27,73,6,69,21,5,5,0,0,0,3,2,1,2,23,27,6,61,72,84,59,80,53,58,50,0,99,4,15,11,0,10,33,0,5,8,0,9,3,2,4,0,1,3,18,6,4,5,19,12,7,1,8,3,5,4,4,10,2,3,11,70,3,1,131,2,14,27,25,6,19,6,3,24,2,15,11,0,0,0,15,29,2,46,5,29,7,1,3,1,5,0,0,4,6,0,3,0,1,1,1,0,15,46,75,14,2,8,95,55,22,12,46,1,1,1,1,1,99,0,0,90,0,8,0,0,2,99,0,99,99,99,9,2,7,2,24,60,5,12,141.0,17,8,3.0,10.0,10.0,12,8.294118,14959

In [13]:
y_train.head()

,TARGET_B,TARGET_D
42043,0,0.0
68510,0,0.0
9940,1,30.0
59275,0,0.0
62638,1,20.0


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8,
                             random_state = 42)

clf.fit(X1_train, y_train['TARGET_B'])

print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X1_train, y_train['TARGET_B'])))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X1_test, y_test['TARGET_B'])))

y_pred = clf.predict(X1_test)
display(y_test['TARGET_B'].value_counts())
display(confusion_matrix(y_test['TARGET_B'], y_pred))

The Accuracy for the Random Forest in the TRAIN set is 0.61
The Accuracy for the Random Forest in the TEST  set is 0.61


1    18254
0    17974
Name: TARGET_B, dtype: int64

array([[11163,  6811],
       [ 7281, 10973]])

### Discuss the model predictions and it's impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How much the money the company will not earn because of missclassifications made by the model?

In [15]:
y

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
16890,1,21.0
60980,1,20.0
48158,1,10.0
54769,1,15.0


When TARGET_B is '1', it means 'cost'. 

Based on the provided results, the Random Forest model achieved an accuracy of approximately 61% both on the training set and the test set. The accuracy indicates the proportion of correctly classified samples out of the total samples. 61% accuracy might seem decent,

True Positives (TP): 11,163

False Positives (FP): 6,811

True Negatives (TN): 10,973

False Negatives (FN): 7,281

The cost of a false positive is slightly less than to the cost of the false negative. This means it is not good news for the comapny. This is because False Negatives (FN) refers to the number of instances where the model incorrectly predicted the negative class as "0" when the true class is "1." The more FN is gotten, the worse for the comapny. 


Total Cost = (Cost of False Positives * Number of False Positives) + (Cost of False Negatives * Number of False Negatives)

In [16]:
y_test['y-pred'] = y_pred

y_test

,TARGET_B,TARGET_D,y-pred
66958,0,0.0,1
11231,1,7.0,1
90375,0,0.0,0
47462,0,0.0,0
81846,0,0.0,0
...,...,...,...
55018,0,0.0,0
31519,0,0.0,1
27487,1,50.0,0
26367,1,20.0,1


In [17]:
FP = y_test[(y_pred == 1) & (y_test['TARGET_B'] == 0)]['TARGET_D'].sum()
FP

0.0

In [18]:
FN = y_test[(y_pred == 0) & (y_test['TARGET_B'] == 1)]['TARGET_D'].sum()
FN = round(FN, 2)
FN

151593.16

#### Answer: 15193.16

### Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. Which one? Please checkout make_scorer alongside with GridSearchCV in order to train the model to maximize the error metric of interest in this case.

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

max_depth_choices = [3,5,7]
min_samples_split_choices = [10, 20, 30]
min_samples_leaf_choices = [5, 10, 15, 20]
max_samples = [0.8]
random_state = [42]

param_grid = {
    'max_depth': max_depth_choices,
    'min_samples_split': min_samples_split_choices,
    'min_samples_leaf': min_samples_leaf_choices,
    'max_samples': max_samples,
    'random_state': random_state
}


model = RandomForestClassifier()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X1_train, y_train['TARGET_B'])

mean_test_scores = grid_search.cv_results_['mean_test_score']
params_list = grid_search.cv_results_['params']

grid_search.best_params_

{'max_depth': 7,
 'max_samples': 0.8,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'random_state': 42}

In [20]:
for score, params in zip(mean_test_scores, params_list):
    print("Mean Test Score:", score)
    print("Hyperparameters:", params)
    print("-------------------")


Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf': 5, 'min_samples_split': 10, 'random_state': 42}
-------------------
Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf': 5, 'min_samples_split': 20, 'random_state': 42}
-------------------
Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf': 5, 'min_samples_split': 30, 'random_state': 42}
-------------------
Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf': 10, 'min_samples_split': 10, 'random_state': 42}
-------------------
Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf': 10, 'min_samples_split': 20, 'random_state': 42}
-------------------
Mean Test Score: 0.5913325512386999
Hyperparameters: {'max_depth': 3, 'max_samples': 0.8, 'min_samples_leaf'

In [23]:
clf2 = RandomForestClassifier(max_depth=7,
                             min_samples_split=10,
                             min_samples_leaf =5,
                             max_samples=0.8,
                             random_state = 42)

clf2.fit(X1_train, y_train['TARGET_B'])

print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(clf2.score(X1_train, y_train['TARGET_B'])))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(clf2.score(X1_test, y_test['TARGET_B'])))

y_pred2 = clf2.predict(X1_test)
display(y_test['TARGET_B'].value_counts())
display(confusion_matrix(y_test['TARGET_B'], y_pred2))

The Accuracy for the Random Forest in the TRAIN set is 0.68
The Accuracy for the Random Forest in the TEST  set is 0.67


1    18254
0    17974
Name: TARGET_B, dtype: int64

array([[11980,  5994],
       [ 6055, 12199]])

As we see, the numbers of False positives and False Negatives are reduced. The accurace rate increased by more than 0.06 in both tests. 